In [3]:
    import cv2

    cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test.mp4")
    r,frame = cap.read()

    points = []


    def draw_line(event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x,y))
            print(f"Points Selected is {(x,y)}")
            if len(points) ==2:
                print(f"Line Drawn from {points[0]} and {points[1]}")
            
    cv2.namedWindow("Select the Points")
    cv2.setMouseCallback("Select the Points",draw_line)
            
            
    while True:
        if not r:
            print("Unable to load the frame.")
            break
        temp_frame = frame.copy()
        if len(points) == 1:
            cv2.circle(temp_frame,points[0],5,(0,255,0),-1)
        elif len(points) == 2:
            cv2.line(temp_frame,points[0],points[1],(0,0,0),1)
        cv2.imshow("Select the Points",temp_frame)
        if cv2.waitKey(1) & 0xff == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

Points Selected is (398, 251)
Points Selected is (397, 561)
Line Drawn from (398, 251) and (397, 561)


In [ ]:
### HEATMAP 1 (OKE HON)
import cv2
import numpy as np
from ultralytics import YOLO
import datetime

# Load model + video
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(3))
h = int(cap.get(4))

out = cv2.VideoWriter("output_heatmap_final1.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w*2, h))

line_x = 399
line_y1, line_y2 = 251, 561
max_people_bar = 20

track_history = {}
in_count = out_count = 0
person_index = {}

def generate_heatmap_from_bboxes(shape, boxes, scale=1.5):
    heatmap = np.zeros((shape[0], shape[1]), dtype=np.float32)
    
    for (x1, y1, x2, y2) in boxes:
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2

        w = (x2 - x1) * scale
        h = (y2 - y1) * scale

        x_grid, y_grid = np.meshgrid(
            np.arange(shape[1]),
            np.arange(shape[0])
        )

        gaussian = np.exp(
            -(((x_grid - cx) ** 2) / (2 * (w/2) ** 2)
            + ((y_grid - cy) ** 2) / (2 * (h/2) ** 2))
        )

        heatmap += gaussian

    heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
    return heatmap


log_counter = 0


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_original = frame.copy()
    centers = []
    bboxes = []  

    results = model.track(frame, persist=True, conf=0.5, classes=[0], tracker="botsort.yaml")

    for r in results:
        for box in r.boxes:
            if box.id is None:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            track_id = int(box.id)

            bboxes.append((x1, y1, x2, y2))  

            if track_id not in person_index:
                person_index[track_id] = len(person_index) + 1

            label = f"PERSON {person_index[track_id]} (ID:{track_id})"

          
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1 - th - 8), (x1 + tw + 4, y1), (0, 0, 0), -1)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            centers.append((cx, cy))

      
            if track_id not in track_history:
                track_history[track_id] = cx
                continue

            prev_cx = track_history[track_id]
            track_history[track_id] = cx

            if prev_cx < line_x <= cx:
                in_count += 1
            elif prev_cx > line_x >= cx:
                out_count += 1

    current_inside = in_count - out_count


    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (w - 10, 170), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.55, frame, 0.45, 0)

    cv2.putText(frame, "HE THONG AI DEM NGUOI RA VAO", (25, 40),
                cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                (w - 300, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

    cv2.line(frame, (line_x, line_y1), (line_x, line_y2), (0, 0, 0), 3)

    cv3 = cv2
    cv2.putText(frame, f"IN: {in_count}", (40, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 4)
    cv2.putText(frame, f"OUT: {out_count}", (40, 155),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 4)
    cv2.putText(frame, f"HIEN TAI: {current_inside}", (300, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

    bar_length = max(0, min(int(current_inside / max_people_bar * 400), 400))
    cv2.rectangle(frame, (300, 150), (700, 170), (80, 80, 80), 2)
    cv2.rectangle(frame, (300, 150), (300 + bar_length, 170), (0, 255, 255), -1)


    heatmap_gray = generate_heatmap_from_bboxes(frame_original.shape[:2], bboxes, scale=1.2)


    heatmap_color = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_TURBO)

    heatmap_overlay = cv2.addWeighted(frame_original, 0.45, heatmap_color, 0.65, 0)

    estimated = len(bboxes) 

 
    cv2.putText(heatmap_overlay, "DENSITY HEATMAP", (20, 50),
                cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 3)
    cv2.putText(heatmap_overlay, f"Est: {estimated}", (20, 110),
                cv2.FONT_HERSHEY_DUPLEX, 2.0, (0, 255, 255), 5)

    combined = np.hstack((frame, heatmap_overlay))



    log_counter += 1
    if log_counter % 30 == 0:
        ratio = current_inside / 18
        status = "VANG" if ratio <= 0.3 else "BINH THUONG" if ratio <= 0.7 else "DONG"
        print(f"[INFO] {datetime.datetime.now().strftime('%H:%M:%S')} | Trong: {current_inside} nguoi → {status}")

    cv2.imshow("HE THONG AI DEM NGUOI + HEATMAP", combined)
    out.write(combined)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("HOÀN TẤT! File đã lưu: output_heatmap_final1.mp4")



0: 384x640 (no detections), 44.0ms
Speed: 11.1ms preprocess, 44.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.1ms
Speed: 2.4ms preprocess, 51.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.8ms
Speed: 2.0ms preprocess, 51.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.2ms
Speed: 2.6ms preprocess, 43.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.3ms
Speed: 2.2ms preprocess, 38.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.5ms
Speed: 2.2ms preprocess, 40.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.4ms
Speed: 1.2ms preprocess, 40.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 45.5ms
Speed: 2.3ms preprocess, 45.5ms inferen

In [ ]:

import cv2
from ultralytics import YOLO
import datetime


model = YOLO("yolov8n.pt")


cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("outputfinal.mp4", fourcc, fps, (int(cap.get(3)), int(cap.get(4))))


line_x = 399         
line_y1, line_y2 = 251, 561

track_history = {}
in_count = 0
out_count = 0

person_index = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        persist=True,
        conf=0.5,
        classes=[0],
        tracker="botsort.yaml"
    )

    if results is None:
        continue

    for r in results:
        for box in r.boxes:
            if box.id is None:
                continue

            x1, y1b, x2, y2b = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1b + y2b) // 2
            track_id = int(box.id)

            if track_id not in person_index:
                person_index[track_id] = len(person_index) + 1

            label_text = f"PERSON {person_index[track_id]} (ID:{track_id})"

            cv2.rectangle(frame, (x1, y1b), (x2, y2b), (0, 255, 0), 2)

            (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1b - th - 8), (x1 + tw + 4, y1b), (0, 0, 0), -1)
            cv2.putText(frame, label_text, (x1, y1b - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            if track_id not in track_history:
                track_history[track_id] = cx
                continue

            prev_cx = track_history[track_id]
            track_history[track_id] = cx

            moved_left_to_right = prev_cx < line_x and cx >= line_x
            moved_right_to_left = prev_cx > line_x and cx <= line_x

            if moved_left_to_right:
                in_count += 1

            elif moved_right_to_left:
                out_count += 1

   
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (frame.shape[1] - 10, 170), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.55, frame, 0.45, 0)

  
    cv2.putText(frame, "HE THONG AI DEM NGUOI RA VAO",
                (25, 40), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)

    cv2.putText(frame, datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                (frame.shape[1] - 300, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

    cv2.line(frame, (line_x, line_y1), (line_x, line_y2), (0, 0, 0), 3)

    cv2.putText(frame, f"IN: {in_count}", (40, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 4)

    cv2.putText(frame, f"OUT: {out_count}", (40, 155),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 4)

    current_inside = in_count - out_count
    cv2.putText(frame, f"HIEN TAI: {current_inside}", (300, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

    bar_start, bar_end = 300, 700
    max_people = 20
    bar_length = int((current_inside / max_people) * (bar_end - bar_start))
    bar_length = max(0, min(bar_length, bar_end - bar_start))

    cv2.rectangle(frame, (bar_start, 150), (bar_end, 170), (80, 80, 80), 2)
    cv2.rectangle(frame, (bar_start, 150), (bar_start + bar_length, 170), (0, 255, 255), -1)
    cv2.imshow("People IN_OUT Counter", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

print("Xong! Da luu file outputfinal.mp4")



0: 384x640 (no detections), 52.4ms
Speed: 15.1ms preprocess, 52.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.7ms
Speed: 2.8ms preprocess, 62.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.2ms
Speed: 3.0ms preprocess, 62.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.9ms
Speed: 2.7ms preprocess, 49.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.7ms
Speed: 2.0ms preprocess, 40.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.7ms
Speed: 1.9ms preprocess, 45.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 41.7ms
Speed: 1.9ms preprocess, 41.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 37.2ms
Speed: 1.4ms preprocess, 37.2ms inferen

In [ ]:

import cv2
import numpy as np
from ultralytics import YOLO
import datetime
import sys 

model = YOLO("yolov8s.pt") 
video_path = r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\mdo.mp4"

cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Tổng số khung hình của video

print(f"--- BẮT ĐẦU XỬ LÝ ---")
print(f"Tổng số frame: {total_frames}")
print(f"Kích thước: {w}x{h} | FPS gốc: {fps}")

out = cv2.VideoWriter("output_heatmap_auto_stop.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

def generate_heatmap_from_bboxes(shape, boxes, scale=1.5):
    heatmap = np.zeros((shape[0], shape[1]), dtype=np.float32)
    for (x1, y1, x2, y2) in boxes:
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        w_box, h_box = (x2 - x1) * scale, (y2 - y1) * scale
        if w_box <= 0 or h_box <= 0: continue
        
        x_grid, y_grid = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
        gaussian = np.exp(-(((x_grid - cx) ** 2) / (2 * (w_box/2) ** 2) + ((y_grid - cy) ** 2) / (2 * (h_box/2) ** 2)))
        heatmap += gaussian
    return np.clip(heatmap * 255, 0, 255).astype(np.uint8)

frame_count = 0

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("\n[THÔNG BÁO] Đã chạy hết video gốc. Đang lưu file...")
        break
    
    frame_count += 1
    if total_frames > 0 and frame_count > total_frames:
        print("\n[THÔNG BÁO] Đã đạt giới hạn frame. Đang lưu file...")
        break

    frame_original = frame.copy()
    bboxes = []

    results = model.track(frame, persist=True, conf=0.25, iou=0.6, classes=[0], tracker="bytetrack.yaml", verbose=False)

    for r in results:
        for box in r.boxes:
            if box.id is None: continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            if (x2 - x1) > 10 and (y2 - y1) > 20:
                bboxes.append((x1, y1, x2, y2))
              
    heatmap_gray = generate_heatmap_from_bboxes(frame_original.shape[:2], bboxes, scale=1.5)
    heatmap_color = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_TURBO)
    combined_frame = cv2.addWeighted(frame_original, 0.6, heatmap_color, 0.4, 0)
    current_people_count = len(bboxes)
    status = "VANG" if current_people_count <= 5 else "DONG (WARNING)" if current_people_count > 12 else "BINH THUONG"
    color_status = (0, 255, 0) if current_people_count <= 5 else (0, 0, 255) if current_people_count > 15 else (0, 255, 255)

    img_h, img_w = combined_frame.shape[:2]


    panel_x, panel_y = 0, 0
   
    panel_w, panel_h = 240, 110 
    
    sub_img = combined_frame.copy()
    
   
    cv2.rectangle(sub_img, (panel_x, panel_y), (panel_x + panel_w, panel_y + panel_h), (0, 0, 0), -1)
    
  
    cv2.rectangle(sub_img, (panel_x, panel_y), (panel_x + 5, panel_y + panel_h), color_status, -1)
    
   
    combined_frame = cv2.addWeighted(sub_img, 0.6, combined_frame, 0.4, 0)

   
    def draw_text_outline(img, text, pos, font, scale, color, thickness):
        x, y = pos
        cv2.putText(img, text, (x, y), font, scale, (0, 0, 0), thickness + 2) # Viền đen
        cv2.putText(img, text, (x, y), font, scale, color, thickness)           # Chữ chính

    draw_text_outline(combined_frame, "CROWD DENSITY AI", (panel_x + 15, panel_y + 30), 
                      cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
    
    draw_text_outline(combined_frame, "EST PEOPLE:", (panel_x + 15, panel_y + 65), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, (180, 180, 180), 1)
 
    draw_text_outline(combined_frame, f"{current_people_count}", (panel_x + 140, panel_y + 65), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    draw_text_outline(combined_frame, "STATUS:", (panel_x + 15, panel_y + 90), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, (180, 180, 180), 1)
   
    draw_text_outline(combined_frame, f"{status}", (panel_x + 95, panel_y + 90), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, color_status, 2)


 
    time_str = datetime.datetime.now().strftime("%H:%M:%S  %d/%m/%Y")
    
    (t_w, t_h), _ = cv2.getTextSize(time_str, cv2.FONT_HERSHEY_SIMPLEX, 0.65, 2)
    time_x = img_w - t_w - 20 
    time_y = 40               

    draw_text_outline(combined_frame, time_str, (time_x, time_y), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
 
    if total_frames > 0:
        progress = frame_count / total_frames
        bar_w = int(progress * w)
        cv2.rectangle(combined_frame, (0, h-10), (w, h), (50, 50, 50), -1) 
        cv2.rectangle(combined_frame, (0, h-10), (bar_w, h), (0, 255, 0), -1) 


    sys.stdout.write(f"\r[Đang xử lý] Frame: {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")
    sys.stdout.flush()

    cv2.imshow("AI HEATMAP (Bam Q de thoat som)", combined_frame)
    out.write(combined_frame)

    if cv2.waitKey(1) == ord('q'):
        print("\n[THÔNG BÁO] Người dùng bấm Q để dừng sớm.")
        break


cap.release()
out.release()
cv2.destroyAllWindows()
print(f"\nHOÀN TẤT! File video đã được lưu tại: output_heatmap_auto_stop.mp4")

--- BẮT ĐẦU XỬ LÝ ---
Tổng số frame: 630
Kích thước: 768x432 | FPS gốc: 30.0
[Đang xử lý] Frame: 630/630 (100.0%)
[THÔNG BÁO] Đã chạy hết video gốc. Đang lưu file...

HOÀN TẤT! File video đã được lưu tại: output_heatmap_auto_stop.mp4
